In [ ]:
%pip install -q -U pip
%pip install -q -U networkx
%pip install -q -U matplotlib
%pip install -q -U numpy

In [ ]:
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
import collections
import numpy as np

from custommodels.Pickle import pickle

In [ ]:
#import data from file
plot_path = "../data/plots/"
path = "../data/loc-brightkite_edges.txt.gz"
G = nx.read_edgelist(path, create_using=nx.Graph(), nodetype=int)
len(G.nodes()), len(G.edges())

#for plots
green = '#94bc24'
navy = '#243444'

# Get largest component

In [ ]:
largest_cc = max(nx.connected_components(G), key=len)
len(largest_cc)
G = G.subgraph(largest_cc)

In [ ]:
print("Number of edges:", len(nx.edges(G)))
print("Number of nodes", len(nx.nodes(G)))
print("The average clustering coefficient for the network:", nx.average_clustering(G))
#print("The clustering coefficient for nodes", nx.clustering(G))
print("The maximum degree of the network:", max(dict(G.degree()).values()))

In [ ]:
# Calculate the degrees of all nodes and store them in a list
degrees = [degree for node, degree in G.degree()]

# Count the frequency of each degree value
degree_count = Counter(degrees)
degree, frequency = zip(*degree_count.items())



# Create a scatter plot of degrees (log-log scale)
plt.scatter(degree, frequency, c=green, edgecolors = navy)

# Set the x and y axis to log scale
plt.xscale('log')
plt.yscale('log')

# Add labels and title
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.title('Degree Distribution on Log-Log Scale')

# save the plot
plt.savefig(plot_path + 'degree_distribution.png')
# Show the plot
plt.show()

Power law


In [ ]:
%pip install -q -U powerlaw

In [ ]:
degree_sequence = sorted([d for n, d in G.degree()], reverse=True)

import powerlaw # Power laws are probability distributions with the form:p(x)∝x−α
fit = powerlaw.Fit(degree_sequence) 
fig2 = fit.plot_pdf(color= navy, linewidth=4)
fit.power_law.plot_pdf(color = green, ax=fig2)
plt.title('Power Law test')
print(fit.power_law.alpha)
plt.savefig(plot_path + 'power_law_test.png')
R, p = fit.distribution_compare('power_law', 'exponential')
print(R, p)
print(degree_sequence)

CCDF log log scale


In [ ]:
# finding the degree CCDF 
def degree_ccdf(graph):
    degree_sequence = sorted([d for n, d in graph.degree()], reverse=True)
    degree_count = collections.Counter(degree_sequence)
    degree, count = zip(*degree_count.items())
    ccdf = 1 - np.cumsum(count[::-1])[::-1]/graph.number_of_nodes()
    return degree, ccdf

# plotting the ccdf for the true graph and the random graph 
degree_G, ccdf_G  = degree_ccdf(G)
plt.figure(figsize=(8, 6))
plt.loglog(degree_G, ccdf_G, marker='o', linestyle='None',  c='#94bc24', mec = '#223747')
plt.title('Degree CCDF in Log-Log Scale')
plt.xlabel('Degree')
plt.ylabel('CCDF')
plt.savefig(plot_path + 'ccdf.png')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import powerlaw as pl
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import linregress
# Load the data and redo the CCDF.
#G = nx.read_edgelist("../4/data.txt", delimiter = "\t")
dd = Counter(dict(G.degree).values())
dd = pd.DataFrame(list(dd.items()), columns = ("k", "count")).sort_values(by = "k")
ccdf = dd.sort_values(by = "k", ascending = False)
ccdf["cumsum"] = ccdf["count"].cumsum()
ccdf["ccdf"] = ccdf["cumsum"] / ccdf["count"].sum()
ccdf = ccdf[["k", "ccdf"]].sort_values(by = "k")
# We take the logarithm in base 10 of both degree and CCDF. Then we simply do a linear regression. The slope is
# the exponent. The intercept needs to be the power of 10, to undo the logarithm operation. Look at that
# r-squared!
logcdf = np.log10(ccdf[["k", "ccdf"]])
slope, log10intercept, r_value, p_value, std_err = linregress(logcdf["k"], logcdf["ccdf"])
#print("CCDF Fit: %1.4f x ^ %1.4f (R2 = %1.4f, p = %1.4f)" % (10 ** log10intercept, slope, r_value ** 2, p_value))

# With the powerlaw package, fitting the CCDf is simple. It will store results in the .power_law property. To
# get the actual k_min, we need to find the degree value corresponding to the probability in .power_law.xmin:
# pandas makes it easy. This is definitely a shifted power law. (Kappa contains the intercept information)
results = pl.Fit(ccdf["ccdf"])
k_min = ccdf[ccdf["ccdf"] == results.power_law.xmin]["k"]
#print("Powerlaw CCDF Fit: %1.4f x ^ -%1.4f (k_min = %d)" % (10 ** results.power_law.Kappa, results.power_law.alpha, k_min))

# Let's plot the best fit.
ccdf["fit"] = (10 ** results.power_law.Kappa) * (ccdf["k"] ** -results.power_law.alpha)
ax = plt.gca()
plt.loglog(degree_G, ccdf_G, marker='o', linestyle='None',  c='#94bc24', mec = '#223747')
#ccdf.plot(y = "ccdf",marker='o', linestyle='None',  c='#94bc24', mec = '#223747', loglog = True, ax = ax)
ccdf.plot(kind = "line", x = "k", y = 'fit', color = '#223747', loglog = True, ax = ax)
ax.legend().set_visible(False) 
plt.title('Degree CCDF in Log-Log Scale')
plt.xlabel('Degree')
plt.ylabel('CCDF')
plt.savefig(plot_path + 'ccdf+line.png')

Density

In [ ]:
density = nx.density(G)
density

Local clustering coefficient

In [ ]:
def clusters():
    clusters = nx.clustering(G)
    return clusters

clusters = pickle(clusters)
sorted_clusters = sorted(clusters.items(), key=lambda item: item[1], reverse = True)


In [ ]:
min(clusters.values()) , max(clusters.values())
# (0,1.0) 
plt.hist(clusters.values(), color = green)
plt.title('Clustering coefficient distribution')
plt.savefig(plot_path + 'clusters.png')
plt.show()
# most nodes have lower clustering coefficients but some also have very high clustering coefficient 

In [ ]:
avg_LCC = sum(clusters.values())/len(clusters.values())
avg_LCC

In [ ]:
# Transitivity = global clustering coefficient
transitivity = nx.transitivity(G)
transitivity

Diameter

In [ ]:
def diameter():
    diameter = nx.diameter(G)
    return diameter

diameter = pickle(diameter)
# answer = 18 
print(diameter)

Average shortest path

In [ ]:
def avg_shortest_path():
    avg_shortest_path = nx.average_shortest_path_length(G)
    return avg_shortest_path
avg_shortest_path = pickle(avg_shortest_path)
avg_shortest_path

# answer: 4.917269741932054

Shortest path distribution 

In [ ]:
# Can't run it, I don't have enough memory - Ania


def shortest_path_lengths():
    # Calculate shortest path lengths
    all_shortest_path_lengths = {}

    for node in G.nodes:
        shortest_path_lengths = nx.single_source_shortest_path_length(G, node)
        all_shortest_path_lengths[node] = shortest_path_lengths
    return all_shortest_path_lengths

# Calculate shortest paths lengths for our grapgh and store in a pickle
shortest_path_lengths = pickle(shortest_path_lengths)
print(shortest_path_lengths)

Betweenness centrality

In [ ]:
def betweenness_centrality():
    betweenness_centrality = nx.betweenness_centrality(G)
    return betweenness_centrality
betweenness_centrality = pickle(betweenness_centrality)
betweenness_centrality


In [ ]:
between_list = []
for key, value in betweenness_centrality.items():
    between_list.append((value))
between_list


In [ ]:
lists = sorted(betweenness_centrality.items()) # sorted by key, return a list of tuples

x, y = zip(*lists) # unpack a list of pairs into two tuples




# Add labels and title
plt.xlabel('Betweenness centrality')
plt.ylabel('Frequency')
plt.title('Betweenness centrality distribution on Log-Log Scale')

plt.hist(y, x, color = green)
plt.show()

Degree distribution dictionary

In [ ]:
def sorted_degree():
    degree_dict = dict(G.degree)
    sorted_degree = (sorted(degree_dict.items(), key=lambda item: item[1], reverse = True))
    return sorted_degree

sorted_degree = pickle(sorted_degree)
print(sorted_degree)